# 第3章 word2vec

## 3.1 基于推理的方法和神经网络

用向量表示单词的研究比较成功的方法大致分两种：
* 基于计数的方法
* 基于推理的方法

两者的背景都是分布式假设。。
分布式假设：单词含义由其周围的单词构成。

### 3.1.1 基于计数的方法的问题

基于计数的方法根据一个单词周围的单词的出现的频数来表示单词。具体来说，先生成所有单词的共现矩阵，
再对这个矩阵进行SVD，以获得密集向量（单词的分布式表示）。但是，基于计数的方法在处
理大规模语料库时会出现问题。



![](../images/图3-1.基于计数的方法和基于推理的方法的比较.PNG)
图3-1　基于计数的方法和基于推理的方法的比较

基于计数的方法一次性处理全部学习数据；
基于推理的方法使用部分学习数据逐步学习。

基于推理的方法的主要操作是“推理”。如图 3-2 所示，当给出周围的
单词（上下文）时，预测“？”处会出现什么单词，这就是推理。
![](../images/图3-2.基于两边的单词（上下文），预测“？”处出现什么单词.PNG)
图3-2.基于两边的单词（上下文），预测“？”处出现什么单词

![](../images/图3-3.基于推理的方法：输入上下文，模型输出各个单词的出现概率.PNG)
图3-3.基于推理的方法：输入上下文，模型输出各个单词的出现概率

如何对基于分布式假设的“单词共现”建模都是最重要的研究主题。

### 3.1.3 神经网络中单词的处理方法

神经网络无法直接处理 you 或 say 这样的单词，要用神经网络处理单词，需要先将单词转化
为固定长度的向量。对此，一种方式是将单词转换为 one-hot 表示（one-hot向量）。

![](../images/图3-4.单词、单词ID以及它们的one-hot表示.PNG)
图3-4.单词、单词ID以及它们的one-hot表示

![](../images/图3-5.输入层的神经元：各个神经元对应于各个单词.PNG)
图3-5.输入层的神经元：各个神经元对应于各个单词。图中神经元为1的地方用黑色绘制，为0的地方用白色绘制


![](../images/图3-6.基于神经网络的全连接层的变换.PNG)
图3-6.基于神经网络的全连接层的变换：输入层的各个神经元分别对应于7个单词（中间层的神经元暂为3个）

![](../images/图3-7.基于全连接层的变换的简化图示.PNG)
图3-7.基于全连接层的变换的简化图示：将全连接层的权重表示为一个7 × 3形状的W矩阵

In [2]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])   # 输入
W = np.random.randn(7, 3)               # 权重
h = np.dot(c, W)                        # 中间节点

print(h)

[[ 0.38944584 -0.92551572 -1.87100278]]


上述代码中的 c 和 W 的矩阵乘积相当于“提取”权重的对应行向量。

![](../images/图3-8.在上下文c和权重W的矩阵乘积中，对应位置的行向量被提取（权重的各个元素.PNG)
图3-8.在上下文c和权重W的矩阵乘积中，对应位置的行向量被提取（权重的各个元素的大小用灰度表示）

In [1]:
# 使用第 1 章中实现的 MatMul 层完成
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.75053228  0.61698931  0.89424161]]


## 3.2 简单的word2vec

CBOW （continuous bag-of-words） 模型和 skip-gram 模型是 word2vec 中使用的两个神经网络。

### 3.2.1 CBOW模型推理

CBOW 模型是根据上下文预测目标词的神经网络（“目标词”是指中间
的单词，它周围的单词是“上下文”）。通过训练这个 CBOW 模型，使其能
尽可能地进行正确的预测，我们可以获得单词的分布式表示。

CBOW 模型的输入是上下文。

![](../images/图3-9.CBOW模型的网络结构.PNG)
图3-9.CBOW模型的网络结构

![](../images/图3-10.权重的各行对应各个单词的分布式表示.PNG)
图3-10.权重的各行对应各个单词的分布式表示

如图 3-10 所示，权重 Win 的各行保存着各个单词的分布式表示。通过
反复学习，不断更新各个单词的分布式表示，以正确地从上下文预测出应当
出现的单词。令人惊讶的是，如此获得的向量很好地对单词含义进行了编
码。这就是 word2vec 的全貌。

![](../images/图3-11.层视角下的CBOW模型的网络结构.PNG)
图3-11.层视角下的CBOW模型的网络结构

参考图 3-11，我们来实现 CBOW 模型的推理（即求得分的过程），具体实现如下所示（[ch03/cbow_predict.py](../ch03/cbow_predict.py)）

### 3.2.2 CBOW模型的学习

![](../images/图3-12.CBOW模型的示例.PNG)
图3-12.CBOW模型的示例（节点值的大小用灰度表示）

CBOW 模型的学习就是调整权重，以使预测准确。

![](../images/图3-13.学习时的CBOW模型的网络结构.PNG)
图3-13.学习时的CBOW模型的网络结构

![](../images/图3-14.将Softmax层和Cross%20Entropy%20Error层统一为Softmax%20with%20Loss层.PNG)
图3-14.将Softmax层和Cross Entropy Error层统一为Softmax with Loss层

### 3.2.3 word2vec的权重和分布式表示

word2vec 中使用的网络有两个权重，分别是输入侧的全连接层的权重（$W_in$）和输出侧的全连接层的权重（$W_out$）。
一般而言，输入侧的权重 $W_in$ 的每一行对应于各个单词的分布式表示。
另外，输出侧的权重 $W_out$ 也同样保存了对单词含义进行了编码的向量。

![](../images/图3-15.输入侧和输出侧的权重都可以被视为单词的分布式表示.PNG)
图3-15.输入侧和输出侧的权重都可以被视为单词的分布式表示

使用哪个权重作为单词的分布式表示呢？这里有三个选项。
* A. 只使用输入侧的权重
* B. 只使用输出侧的权重
* C. 同时使用两个权重

就 word2vec（特别是 skip-gram 模型）而言，最受欢迎的是方案 A。

## 3.3 学习数据的准备

“You say goodbye and I say hello.”

### 3.3.1 上下文和目标词

word2vec 中使用的神经网络的输入是上下文，它的正确解标签是被这
些上下文包围在中间的单词，即目标词。也就是说，我们要做的事情是，当
向神经网络输入上下文时，使目标词出现的概率高（为了达成这一目标而进
行学习）。

![](../images/图3-16.从语料库生成上下文和目标词.PNG)
图3-16.从语料库生成上下文和目标词

In [4]:
# 将语料库的文本转化成单词 ID
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus) # [0 1 2 3 4 1 5 6]
print(id_to_word) # {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}



![](../images/图3-17.从单词ID列表corpus生成contexts和target的例子.PNG)
图3-17.从单词ID列表corpus生成contexts和target的例子

生成上下文和目标词的函数（create_contexts_target(corpus, window_size)（[common/util.py](../common/util.py)））


In [5]:
from common.util import create_contexts_target
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]


In [6]:
print(target)

[1 2 3 4 1 5]


### 3.3.2 转化为one-hot表示

![](../images/图3-18.将上下文和目标词转化为one-hot表示的例子.PNG)
图3-18.将上下文和目标词转化为one-hot表示的例子


convert_one_hot() 函数以将单词 ID 转化为 one-hot 表示（代码在 [common/util.py](../common/util.py) 中）


In [2]:
import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
contexts, target = create_contexts_target(corpus, window_size=1)
vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

target, contexts

(array([[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0]]),
 array([[[1, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0]],
 
        [[0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0]],
 
        [[0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0]],
 
        [[0, 0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0]],
 
        [[0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0]],
 
        [[0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1]]]))

## 3.4 CBOW模型的实现

![](../images/图3-19.CBOW模型的网络结构.PNG)
图3-19.CBOW模型的网络结构

图 3-19 中的神经网络实现为 SimpleCBOW 类（[ch03/simple_cbow.py](../ch03/simple_cbow.py)）

![](../images/图3-20.CBOW模型的反向传播：蓝色的粗线表示反向传播的路线.PNG)
图3-20.CBOW模型的反向传播：蓝色的粗线表示反向传播的路线

### 学习的实现

（[ch03/train.py](../ch03/train.py)）

![](../images/图3-21.用图形表示学习过程（横轴表示学习的迭代次数，纵轴表示损失）.PNG)
图3-21.用图形表示学习过程（横轴表示学习的迭代次数，纵轴表示损失）

## 3.5 word2vec的补充说明

从概率的角度，再来看一下 CBOW 模型。

### 3.5.1 CBOW模型和概率

CBOW模型进行的处理是，当给定某个上下文时，输出目标词的概率。

![](../images/图3-22.word2vec的CBOW模型：从上下文的单词预测目标词.PNG)
图3-22.word2vec的CBOW模型：从上下文的单词预测目标词

$$
    P(w_t \mid w_{t-1}, w_{t+1}) \tag{3.1}
$$

式 (3.1) 表示“在 $w_{t−1}$ 和 $w_{t+1}$ 发生后，$w_t$ 发生的概率”，也可以解释
为“当给定 $w_{t−1}$ 和 $w_{t+1}$ 时，$w_t$ 发生的概率”。也就是说，CBOW 模型可
以建模为式 (3.1)。

使用式 (3.1) 可以简洁地表示 CBOW 模型的损失函数。

交叉熵误差函数：
$$
    L = -\sum_k t_k \log y_k \tag{1.7}
$$
$y_k$ 是第 $k$ 个事件发生的概率，
$t_k$ 是第 $k$ 是监督标签，是one-hot向量的元素。

$$
    L = - \log P(w_t \mid w_{t-1}, w_{t+1}) \tag{3.2}
$$

CBOW 模型的损失函数只是对式 (3.1) 的概率取 log，并加上负号。顺
便提一下，这也称为**负对数似然**（negative log likelihood）。

式 (3.2) 是一笔样本数据的损失函数。如果将其扩展到整个语料库，则损失函数可以写为：
$$
    L = - \frac{1}{T} \sum_{t=1}^T \log P(w_t \mid w_{t-1}, w_{t+1}) \tag{3.3}
$$

CBOW 模型学习的任务就是让式 (3.3) 表示的损失函数尽可能地小。

### 3.5.2 skip-gram模型

word2vec 有两个模型：一个是我们已经讨论过的 CBOW模型；另一个是被称为 skip-gram 的模型。

skip-gram 是反转了 CBOW 模型处理的上下文和目标词的模型。

![](../images/图3-23.CBOW模型和skip-gram模型处理的问题.PNG)
图3-23.CBOW模型和skip-gram模型处理的问题

![](../images/图3-24.skip-gram模型的例子.PNG)
图3-24.skip-gram模型的例子

$$
    P(w_{t-1},w_{t+1} \mid w_t) \tag{3.4}
$$

$$
    P(w_{t-1},w_{t+1} \mid w_t) = P(w_{t-1} \mid w_t)P(w{t+1} \mid w_t) \tag{3.5}
$$

通过将式 (3.5) 代入交叉熵误差函数，可以推导出 skip-gram 模型的损失函数：
$$
    L = - \log P(w_{t-1}, w_{t+1} \mid w_t) \\
        = - \log P(w_{t-1} \mid w_t) P(w_{t+1} \mid w_t) \\
        = - (\log P(w_{t-1} \mid w_t) + \log P(w_{t+1} \mid w_t)) \\
        \tag{3.6}
$$

如果扩展到整个语料库，则skip-gram 模型的损失函数可以表示为式 (3.7)：

$$
    L = - \frac{1}{T} \sum^T_{t=1}(\log P(w_{t-1} \mid w_t) + \log P(w_{t+1} \mid w_t)) \\
        \tag{3.7}
$$

我们应该使用 CBOW 模型和 skip-gram 模型中的哪一个呢？答
案应该是 skip-gram 模型。这是因为，从单词的分布式表示的准确度来看，
在大多数情况下，skip-grm 模型的结果更好。特别是随着语料库规模的增
大，在低频词和类推问题的性能方面，skip-gram 模型往往会有更好的表现
（单词的分布式表示的评价方法会在 4.4.2 节说明）。此外，就学习速度而言，
CBOW 模型比 skip-gram 模型要快。这是因为 skip-gram 模型需要根据上
下文数量计算相应个数的损失，计算成本变大。

skip-gram 模型的实现[ch03/simple_skip_gram.py](../ch03/simple_skip_gram.py)

### 3.5.3 基于计数与基于推理

基于计数的方法通过对整个语料库的统计数据进行一次学习来获得单词的分布式表示，
基于推理的方法则反复观察语料库的一部分数据进行学习（mini-batch学习）。



就单词相似性的定量评价而言，基于推理的方法和基于计数的方法难分上下。


基于推理的方法和基于计数的方法存在关联性。
使用了 skip-gram 和下一章介绍的 Negative Sampling 的模
型被证明与对整个语料库的共现矩阵（实际上会对矩阵进行一定的修改）进
行特殊矩阵分解的方法具有相同的作用。

## 本章所学的内容

* 基于推理的方法以预测为目标，同时获得了作为副产物的分布式表示
* word2vec是基于推理的方法，由简单的2层神经网络构成
* word2vec有skip-gram模型和CBOW模型
* CBOW模型从多个单词（上下文）预测1个个单词（目标词）
* skip-gram模型反过来从1个单词（目标词）预测多个单词（上下文）
* 由于word2vec可以进行权重的增量学习，所以能够高效地更新或添加单词的分布式表示